In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import sys
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale

from sklearn import metrics
import pandas as pd
from glob import glob
import pickle
from sleep_eval import evaluate_scoring_algorithm, evaluation_summary
from sleep_misc import rescore_models
from sleep_summary import load_results
#from scipy.stats import ttest_ind


In [ ]:
TASK=2
df = load_results(TASK)
df = df.fillna(0.0)

if TASK == 1:
    best = "LSTM 100"
else:
    best = "CNN 100"

del df["ExtraTrees"]
key_mapping = {"p_sazonov":"Sazonov", 
               "p_cole":"Cole-Kripke",
               "p_sadeh":"Sadeh",
               "p_webster": "Webster",
               "p_oakley" : "Oakley",
               "p_kripke" : "Scripps Clinic",
               "p_SGD_log" : "Logistic Reg.",
               "p_ExtraTrees" : "ExtraTrees",
               "p_LSTM_20_raw": "LSTM 20",
               "p_LSTM_50_raw": "LSTM 50",
               "p_LSTM_100_raw": "LSTM 100",
               "p_CNN_20_raw": "CNN 20",
               "p_CNN_50_raw": "CNN 50",
               "p_CNN_100_raw": "CNN 100",
              }
df = df.rename(columns=key_mapping)
df.head()


In [ ]:
"""
    Make AU_ROC and AU_PR plots for top methods
"""

invert_label = ["Cole-Kripke", "Webster", "Oakley", "Scripps Clinic"]

plt.figure(figsize=(16,10))
models = ['Sazonov', 'Cole-Kripke','Sadeh', 'Oakley', 'Scripps Clinic', 'Webster', 
          'Logistic Reg.', 'ExtraTrees',
          'LSTM 20', 'LSTM 50', 'LSTM 100', 'CNN 20', 'CNN 50', 'CNN 100']
linestyles = ['-', '--', '-.', ':']
params = {'axes.titlesize': 24,
          'axes.labelsize': 24,
          'lines.markersize': 14,
          'xtick.labelsize' : 24,
          'ytick.labelsize' : 24,
          'legend.fontsize': 20,
          'legend.handlelength': 2}

plt.rcParams.update(params)
for (i, m) in enumerate(models):
    pred = minmax_scale(df[m].values)
    label = df['gt']
    if (m in invert_label):
        label = label==0
    fpr, tpr, thresh = metrics.roc_curve(label, pred)
    auc = "%.3f" % metrics.roc_auc_score(label, pred)
    auc = auc.lstrip("0")
    
    if m == best:
        l = "%s - AUROC = %s" % (m, auc)
        l = ' '.join(['$\\bf{'+j+'}$' for j in l.split(' ')])
        plt.plot(fpr, tpr, label=l, linewidth=3, linestyle=linestyles[i%4])
    else:
        plt.plot(fpr,tpr,label='%s - AUROC = %s' % (m, auc),linewidth=3.0, linestyle = linestyles[i%4])

ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(True)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(True)  
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   

plt.legend(loc="lower right", fancybox=True, framealpha=0.5)
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('AUROC curve for Task %d' % (TASK))
plt.show()


In [ ]:
plt.figure(figsize=(16,10))
models = ['Sazonov', 'Sadeh', 'oakley', 'cole', 'kripke', "webster",
#          'Logistic Reg.',  'Perceptron', 'Linear SVM', 'ExtraTrees',
          'LSTM 20', 'LSTM 50', 'LSTM 100', 'CNN 20', 'CNN 50', 'CNN 100']
linestyles = ['-', '--', '-.', ':']
params = {'axes.titlesize': 24,
          'axes.labelsize': 24,
          'lines.markersize': 14,
          'xtick.labelsize' : 24,
          'ytick.labelsize' : 24,
          'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.rcParams.update(params)
for (i, m) in enumerate(models):
    pred = minmax_scale(df[m].values)
    label = df['gt']
    if (m in invert_label):
        label = label==0
    if m in ["oakley", "cole", "kripke", 'webster']:
        mapping = {"cole":"Cole-Kripke", "oakley" : "Oakley", "kripke" : "Scripps Clinic", "webster":"Webster"}
        m = mapping[m]
    prec, rec, thresh = metrics.precision_recall_curve(label, pred)
    auc = "%.3f" % metrics.auc(rec,prec)
    auc = auc.lstrip("0")
    if m == best:
        l = "%s - AUPS = %s" % (m, auc)
        l = ' '.join(['$\\bf{'+j+'}$' for j in l.split(' ')])
        plt.plot(rec, prec, label=l, linewidth=3, linestyle=linestyles[i%4])
    else:
        plt.plot(rec, prec, label="%s - AUPS = %s" % (m, auc), linewidth=3, linestyle=linestyles[i%4])

#ax = plt.subplot(111)
#box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(True)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(True)  
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left() 

plt.legend(loc="lower center",  fancybox=True, framealpha=0.5)

plt.xlabel('Sensitivity')
plt.ylabel('Precision')
plt.title('AUPS curve for Task %d' % (TASK))
plt.show()
